In [49]:
from scipy import interpolate
import numpy as np
import h5py
import pandas as pd
from osgeo import gdal, osr
#import gdal, osr
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import datetime

## Read S2 SRF from csv file and prepare S2 SRF array

In [50]:
df = pd.read_csv('S2-SRF_COPE-GSEG-EOPG_S2B.csv')

In [51]:
S2_B= {}
for i in  range (1,len(df.columns)):   
    S2_B[i-1]= list(zip(df.loc[:,df.columns[0]],df.loc[:,df.columns[i]]))
    S2_B[i-1] = {key:val for key, val in list(S2_B[i-1]) if val != 0.0} ## remove all 0 in the value

## Load NEON and extract parameters from h5 file

In [52]:
fullfileName='E:/NEON_refl-surf-dir-ortho-lin0e/NEON_refl-surf-dir-ortho-line/NEON.D16.ABBY.DP1.30006.001.2021-07.basic.20221124T030111Z.PROVISIONAL/NEON_D16_ABBY_DP1_20210719_191207_reflectance.h5'
file_base=os.path.basename(fullfileName)
file_name=os.path.splitext(file_base)[0]
# print (file_name)
file_1m=file_name+'.tif' ## output file name
file_10m=file_name+'_10m.tif' ## output file name

f=h5py.File(fullfileName,rdcc_nbytes =1024**2*10000,rdcc_nslots=1e7 )

In [30]:
# f=h5py.File('K:/NEON_refl-surf-dir-ortho-line (3)/NEON_refl-surf-dir-ortho-line/NEON.D16.ABBY.DP1.30006.001.2019-07.basic.20221110T005401Z.RELEASE-2022/NEON_D16_ABBY_DP1_20190714_220751_reflectance.h5','r')

In [31]:
# f=h5py.File('E:/NEON_refl-surf-dir-ortho-lin0e/NEON_refl-surf-dir-ortho-line/NEON.D16.ABBY.DP1.30006.001.2021-07.basic.20221124T030111Z.PROVISIONAL/NEON_D16_ABBY_DP1_20210724_163435_reflectance.h5',rdcc_nbytes =1024**2*10000,rdcc_nslots=1e7 )

In [32]:
# f=h5py.File('E:/NEON_refl-surf-dir-ortho-lin0e/NEON_refl-surf-dir-ortho-line/NEON.D16.ABBY.DP1.30006.001.2021-07.basic.20221124T030111Z.PROVISIONAL/NEON_D16_ABBY_DP1_20210719_191207_reflectance.h5',rdcc_nbytes =1024**2*10000,rdcc_nslots=1e7 )

In [53]:
site='ABBY'  ## site name
S2_Bands=13  ## the band number of S2 bands 

In [54]:
Ref = f[site]['Reflectance']
RefArray = Ref['Reflectance_Data']
Ref_shape = RefArray.shape
MapInfo = Ref['Metadata']['Coordinate_System']['Map_Info']
epsg_code = int(Ref['Metadata']['Coordinate_System']['EPSG Code'][()]) ### epsg, convert byte to int
noDataValue =RefArray.attrs['Data_Ignore_Value']
scaleFactor = RefArray.attrs['Scale_Factor']
wavelengths = Ref['Metadata']['Spectral_Data']['Wavelength']
neonwave=wavelengths[()] ## NEON wavelength
flighttime = Ref['Metadata']['Flight_Trajectory']['Flight_Time']

## Prepare angle parameters

In [55]:
Solar_Az = Ref['Metadata']['Logs']['Solar_Azimuth_Angle'][()] ## The solar azimuth angle , single value
Solar_Zen = Ref['Metadata']['Logs']['Solar_Zenith_Angle'][()] ## The solar zenith angle , single value
ToSensor_Zen= Ref['Metadata']['to-sensor_Zenith_Angle']   ##  The to-sensor zenith angle from the platform, array
ToSensor_Az = Ref['Metadata']['to-sensor_Azimuth_Angle'] ## The to-sensor azimuth angle from the platform,array

In [56]:
ToSensor_Az_np=np.asarray(ToSensor_Az) ### from h5 to numpy array
ToSensor_Zen_np=np.asarray(ToSensor_Zen) ### from h5 to numpy array
ToSensor_Az_np[ToSensor_Az_np==-9999.]=np.nan ## assign nan to -9999.  
ToSensor_Zen_np[ToSensor_Zen_np==-9999.]=np.nan ## assign nan to -9999.
# ToSensor_Az_np=np.radians(ToSensor_Az_np) ## convert to radians
ToSensor_Zen_np=np.radians(ToSensor_Zen_np) ## convert to radians

In [57]:
cosVZA=np.cos(ToSensor_Zen_np)  ##vza: 'MEAN_INCIDENCE_ZENITH_ANGLE',
Solar_Zen_np=np.copy(ToSensor_Zen_np) ### get a copy of angle file with same nan
Solar_Zen_np[~np.isnan(Solar_Zen_np)]=Solar_Zen  ### assign a single value solar zenith angle to the array not nan
Solar_Zen_np=np.radians(Solar_Zen_np) ## convert to radian
cosSZA=np.cos(Solar_Zen_np)  ## sza: 'MEAN_SOLAR_ZENITH_ANGLE',
Solar_Az_np=np.copy(ToSensor_Az_np) ### get a copy of angle file with same nan
Solar_Az_np[~np.isnan(Solar_Az_np)]=Solar_Az  ### assign a single value solar azimuth angle to the array not nan
# cosRAA=np.cos(np.abs(np.subtract(Solar_Az_np,ToSensor_Az_np)))
cosRAA=np.cos(np.radians(np.abs(np.subtract(Solar_Az_np,ToSensor_Az_np))))

In [58]:
angles=np.stack((cosVZA,cosSZA,cosRAA), axis=0) ## stack angles

## interpolate S2 from Nenon wavelenth

In [59]:
neon_S2_B=np.zeros(shape=(13,426))

In [60]:
## S2 13 bands, index from 0 to 12, interpolate S2 SRF for each NEON spectral band
for i in range(13):
    neon_S2_B[i]=np.interp(neonwave,np.fromiter(S2_B[i].keys(), dtype=float), np.fromiter(S2_B[i].values(), dtype=float), 0, 0)

In [61]:
# Sim_Output=np.zeros(shape=(Ref_shape[0],Ref_shape[1],S2_Bands))
Sim_Output=np.zeros(shape=(S2_Bands,Ref_shape[0],Ref_shape[1]))

In [62]:
t1=datetime.now()
if (Ref_shape[1]>Ref_shape[0]):   ## 'width>height')
    # print ('width>height')
    for j in range(Ref_shape[1]):
    # if (j%100==0):
        # print ('j=',j)
        temp=RefArray[:,j,:]# get one row of the array to make the process faster
        for i in range(Ref_shape[0]):
            # if (j%100==0):
            #     print ('j=',j)
            # pixel_vec = RefArray[i,j,:].astype(np.float); ## get the pixel vector for data cube
            pixel_vec = temp[i,:].astype(np.float); ## get the pixel vector for data cube
            pixel_vec[pixel_vec==int(noDataValue)]=np.nan; ## assign np.nan to no data 
            pixel_vec=pixel_vec/scaleFactor; ## divide by scalefactor 
            for k in range(S2_Bands):            
                # Sim_Output[i][j][k]=np.dot(pixel_vec,neon_S2_B[k])/(np.count_nonzero(neon_S2_B[k])); ### pixel_vec dot production with SRF, divided by the number of bands falling in this band seperacal range.
                 Sim_Output[k][i][j]=np.dot(pixel_vec,neon_S2_B[k])/(np.count_nonzero(neon_S2_B[k])); ### pixel_vec dot production with SRF, divided by the number of bands falling in this band seperacal range.
else: ## 'height>width')
    # print ('height>width')
    for i in range(Ref_shape[0]):
        temp=RefArray[i,:,:]# get one row of the array to make the process faster
        for j in range(Ref_shape[1]):       
            #pixel_vec = RefArray[i,j,:].astype(np.float); ## get the pixel vector for data cube
            pixel_vec = temp[j,:].astype(np.float); ## get the pixel vector for data cube
            pixel_vec[pixel_vec==int(noDataValue)]=np.nan; ## assign np.nan to no data 
            pixel_vec=pixel_vec/scaleFactor; ## divide by scalefactor 
            for k in range(S2_Bands):            
                Sim_Output[k][i][j]=np.dot(pixel_vec,neon_S2_B[k])/(np.count_nonzero(neon_S2_B[k])); ### pixel_vec dot production with SRF, divided by the number of bands falling in this band seperacal range.
t2=datetime.now()         
print (t2-t1)        

0:16:38.054559


In [63]:
Result_output=np.append(Sim_Output, angles, axis=0)  ### merge band results and angles

In [64]:
def CreateGeoTiff(Name, Array, driver, NDV, 
                  GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    # DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    # DataSet = driver.Create(Name, Array.shape[1], Array.shape[0], Array.shape[2], DataType)  ## col, row, bands
    DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)  ## col, row, bands
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection( Projection.ExportToWkt() )
    # for i in range(Array.shape[2]):
    for i in range(Array.shape[0]):       
        # DataSet.GetRasterBand(i+1).WriteArray(Array[:,:,i] )
        DataSet.GetRasterBand(i+1).WriteArray(Array[i,:,:] )
        DataSet.GetRasterBand(i+1).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

## Prepare coordinate information for outputting file

In [65]:
MapInfo_string = str(MapInfo[()]) #convert to string
MapInfo_split = MapInfo_string.split(",") #split the strings using the separator "," 
print(MapInfo_split)
xres = float(MapInfo_split[5])
yres=float(MapInfo_split[6])
print('Resolution:',xres,yres)
#Extract the upper left-hand corner coordinates from mapInfo
xMin = float(MapInfo_split[3]) 
yMax = float(MapInfo_split[4])

["b'UTM", '  1.000', '  1.000', '  551696.000', '  5073247.000', '  1.0000000000e+000', '  1.0000000000e+000', '  10', '  North', '  WGS-84', '  units=Meters', " 0'"]
Resolution: 1.0 1.0


In [66]:
## set the projection information 
srs = osr.SpatialReference()
# srs.ImportFromEPSG(32610)  
srs.ImportFromEPSG(epsg_code) 
geotransform=(xMin,xres,0,yMax,0, -yres) 
data_type = gdal.GDT_Float32
driver = gdal.GetDriverByName('GTiff')

In [67]:
# CreateGeoTiff('test6_withangles.tif', Sim_Output, driver, -9999, geotransform, srs, data_type)
# CreateGeoTiff('ABBY_DP1_20210724_163435.tif', Result_output, driver, -9999, geotransform, srs, data_type)
CreateGeoTiff(file_1m, Result_output, driver, -9999, geotransform, srs, data_type)

'NEON_D16_ABBY_DP1_20210719_191207_reflectance.tif'

In [68]:
# raster_rprj = gdal.Warp(outfn, infn, xRes = 10, yRes = 10, resampleAlg = "near")
# raster_rprj = gdal.Warp(outfn, infn, xRes=10, yRes=10, resampleAlg = "average")
raster_rprj = gdal.Warp(file_10m, file_1m, xRes=10, yRes=10, resampleAlg = "average")
raster_rprj = None

In [ ]:
# fn = r'H:\neon\imgreg\S2_ABBY_20210719_1912073.tif'
# ds = gdal.Open(fn, 1)

In [ ]:
# ds.GetGeoTransform()

In [ ]:
# ds.GetProjection()

In [ ]:
# proj = osr.SpatialReference(wkt=ds.GetProjection())
# print(proj.GetAttrValue('AUTHORITY',1))

In [ ]:
# epsg_num=osr.SpatialReference(wkt=ds.GetProjection()).GetAttrValue('AUTHORITY',1)